# Ricci rotation coefficents and Riemann tensor and frame bundle

## Basic initialization of the lorentzian manifold

Initialization of the Schwarzschild manifold $\mathcal M_{Schwarzschild}$ atlas with Schwarzschild chart (SD) and of the metric tensor $g$ :

In [1]:
%display latex

M = Manifold(4, 'M', latex_name=r'\mathcal{M}', structure='Lorentzian')

SD.<t, r, th, ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi:periodic")
var('m', domain='real')

g = M.metric() #we put the label g on the metric tensor

g[0, 0] = - (1 - 2*m/r)
g[1, 1] = 1/(1 - 2*m/r)
g[2, 2] = r^2
g[3, 3] = r^2*sin(th)^2

g.display()

g = (2*m/r - 1) dt*dt - 1/(2*m/r - 1) dr*dr + r^2 dth*dth + r^2*sin(th)^2 dph*dph

Definition of the Levi Civita connection $\nabla$:

In [2]:
nabla = g.connection()

Definition of Ricci $R_{ab}$ and Riemann $R^{a}_{bcd}$ tensors:

In [3]:
Ric_dd = g.ricci()
Riem_uddd = g.riemann()

Definition of the fully covariant Riemann tensor $R_{abcd}$ :

In [4]:
Riem_dddd = Riem_uddd.down(g)
Riem_dddd = Riem_dddd.antisymmetrize(0,1) #it is already antisimmetric in the first two indices, but i make it explicit for Sage
Riem_dddd.set_name('R')

## Initialization of the orthonormal tetrad

Definition of the orthonormal tetrad frame $e_{(a)}$ (Chandrasekar notation) and its dual frame $e^{(a)} = \eta^{(a)(b)}g(e_{(b)},-)$ :

In [5]:
e0 = M.vector_field(1/sqrt(1-2*m/r), 0, 0, 0, frame=SD.frame(), name='e_{(0)}')
e1 = M.vector_field(0, sqrt(1-2*m/r), 0, 0, frame=SD.frame(), name='e_{(1)}')
e2 = M.vector_field(0, 0, 1/r, 0, frame=SD.frame(), name='e_{(2)}')
e3 = M.vector_field(0, 0, 0, 1/(r*sin(th)), frame=SD.frame(), name='e_{(3)}')


e = M.vector_frame('e', (e0, e1, e2, e3)) #costruction of the tetrad collection object e_{a}
de = e.coframe() #construction of the tetrad coframe object e^{a}      

Orthonormality check $g(e_{(a)},e_{(b)}) = \eta_{(a)(b)}$ :

In [6]:
from sage.tensor.modules.comp import Components

scalar_field_algebra = M.scalar_field_algebra()

#definition of eta_{(a)(b)}
eta = Components(scalar_field_algebra, e, 2)
eta[0,0] = M.scalar_field({SD: -1}, name='-1')
eta[1,1] = M.scalar_field({SD: 1}, name='1')
eta[2,2] = M.scalar_field({SD: 1}, name='1')
eta[3,3] = M.scalar_field({SD: 1}, name='1')

for i in range(0,4):
    for j in range(0,4):
        if (g(e[i],e[j]) != eta[i,j]):
            print("error")

## Ricci rotation coefficents

Ricci rotation coefficents $\omega^{(a)}_{\ \ \ \ (b)(c)} = e^{(a)}_{\ \ \ \ \nu}\nabla_{\mu} e_{(b)}^{\ \ \ \ \nu} e_{(c)}^{\ \ \ \ \mu}$ :

In [7]:
#Ricci rotation coefficents as a python class
class Ricci_rotation_coefficents:
    def __init__(self,Gamma):
        self.coef = Gamma
        self.tindices = ["up","down","down"]
        
    def __repr__(self):
        return self.coef[:]
    
    def copy(self):
        copy_ = Ricci_rotation_coefficents(self.coef.copy())
        copy_.tindices = list(self.tindices)
        return copy_
    
    def info(self):
        return f"Indices position: {self.tindices}"
        
    #def __getitem__(self,*indices):
     #   return self.coef[indices]
        
    def contract_with_eta(self,omega_,eta_,pos_index_,typ):
        if(pos_index_ == 1):
                omega_.coef = self.coef.contract(0,eta,0) #the contraction results in omega^{a}_{bc}*eta_{ad}=obj_{bcd} -> indices has to be permutated in dbc to have the correct form
                omega_.coef = omega_.coef.swap_adjacent_indices(0,2,3) #scambia (bc) con (d)
                omega_.tindices[0] = typ
        elif(pos_index_ == 2):
                omega_.coef = self.coef.contract(1,eta,0) #the contraction results in omega_{a}^{b}_{c}*eta_{bd}=obj_{acd} -> indices has to be permutated in dbc to have the correct form
                omega_.coef = omega_.coef.swap_adjacent_indices(1,2,3) #swaps (c) with (d)
                omega_.tindices[1] = typ
        elif(pos_index_ == 3):
                omega_.coef = self.coef.contract(2,eta,0) #the contraction results in omega_{ab}^{c}*eta_{cd}=obj_{abd} -> indices are already ok
                omega_.tindices[2] = typ
        
    def down(self,pos_index):
        if (self.tindices[pos_index-1] == "down"):
            print("error, index already down")
        else:
            omega_down = self.copy()
            self.contract_with_eta(omega_down,eta,pos_index,"down")
            return omega_down
        
    def up(self,pos_index):
        if (self.tindices[pos_index-1] == "up"):
            print("error, index already up")
        else:
            omega_up = self.copy()
            self.contract_with_eta(omega_up,eta,pos_index,"up")
            return omega_up
        

Calculation of $\omega^{(a)}_{\ \ \ \ (b)(c)}$ from $D(e_{(c)},e_{(b)})=\omega^{(a)}_{\ \ \ \ (b)(c)}e_{(a)}$, where D is the Levi Civita connection:

In [8]:
omega_udd = Ricci_rotation_coefficents(nabla.coef(e))

Calculation of all the possible tetrad indices placements of the rotation coefficents with the methods up() and down():

In [9]:
omega_ddd = omega_udd.down(1)
omega_dud = omega_ddd.up(2)
omega_ddu = omega_ddd.up(3)
omega_uud = omega_udd.up(2)
omega_udu = omega_ddu.up(1)
omega_uuu = omega_uud.up(3)
omega_duu = omega_uuu.down(1)

Check of the position of the indices with the method info():

In [10]:
print(omega_ddd.info())
print(omega_udd.info())
print(omega_dud.info())
print(omega_ddu.info())
print(omega_uud.info())
print(omega_udu.info())
print(omega_duu.info())
print(omega_uuu.info())

Indices position: ['down', 'down', 'down']
Indices position: ['up', 'down', 'down']
Indices position: ['down', 'up', 'down']
Indices position: ['down', 'down', 'up']
Indices position: ['up', 'up', 'down']
Indices position: ['up', 'down', 'up']
Indices position: ['down', 'up', 'up']
Indices position: ['up', 'up', 'up']


Check of $\omega_{(a)(b)(c)} = -\omega_{(b)(a)(c)}$ :

In [11]:
omega_ddd.coef == -omega_ddd.coef.swap_adjacent_indices(0,1,2)

True

Check of $\omega_{(a)(b)}^{\ \ \ \ \ \ \ \ \ (c)} = -\omega_{(b)(a)}^{\ \ \ \ \ \ \ \ \ (c)}$ :

In [12]:
omega_ddu.coef == -omega_ddu.coef.swap_adjacent_indices(0,1,2)

True

Check of $\omega^{(a)(b)(c)} = -\omega^{(b)(a)(c)}$ :

In [13]:
omega_uuu.coef == -omega_uuu.coef.swap_adjacent_indices(0,1,2)

True

Check of $\omega^{(a)(b)}_{\ \ \ \ \ \ \ \ \ (c)} = -\omega^{(b)(a)}_{\ \ \ \ \ \ \ \ \ (c)}$ :

In [14]:
 omega_uud.coef == -omega_uud.coef.swap_adjacent_indices(0,1,2)

True

## Riemann tensor tetrad components from Ricci rotation coefficents

Calculation of $\omega_{(a)(b)(c),(d)} = \omega_{(a)(b)(c),\mu}e_{(d)}^{\ \ \ \ \mu}$ :

In [15]:
d_omega_ddd_d = Components(scalar_field_algebra, e, 4)
for a in range(0,4):
    for b in range(0,4):
        for c in range(0,4):
            for d in range(0,4):
                d_omega_form = nabla(M.scalar_field({SD: omega_ddd.coef[a,b,c]}))
                d_omega_ddd_d[a,b,c,d] = d_omega_form(e[d])


Calculation of $\omega_{(b)(a)(f)}\omega_{(c) \ \ \ \ (d)}^{\ \ \ \ (f)}$ and $\omega_{(b)(a)(f)}\omega_{(d) \ \ \ \ (c)}^{\ \ \ \ (f)}$ :

In [16]:
contr1 = omega_ddd.coef.swap_adjacent_indices(0,1,2).contract(2,omega_dud.coef,1)
contr2 = contr1.swap_adjacent_indices(2,3,4)

Calculation of $\omega_{(f)(a)(c)}\omega_{(b) \ \ \ \ (d)}^{\ \ \ \ (f)}$ and $\omega_{(f)(a)(d)}\omega_{(b) \ \ \ \ (c)}^{\ \ \ \ (f)}$ :

In [17]:
contr3_4 = omega_ddd.coef.contract(0,omega_dud.coef,1)
contr3 = contr3_4.swap_adjacent_indices(1,2,3)
contr4 = contr3_4.swap_adjacent_indices(1,2,4)

Finally we compute $R_{(a)(b)(c)(d)} = -\omega_{(a)(b)(c),(d)}+\omega_{(a)(b)(d),(c)}+\omega_{(b)(a)(f)}\left(\omega_{(c) \ \ \ \ (d)}^{\ \ \ \ (f)}-\omega_{(d) \ \ \ \ (c)}^{\ \ \ \ (f)}\right) + \omega_{(f)(a)(c)}\omega_{(b) \ \ \ \ (d)}^{\ \ \ \ (f)} - \omega_{(f)(a)(d)}\omega_{(b) \ \ \ \ (c)}^{\ \ \ \ (f)}$

In [18]:
Riem_tetrad_dddd = -d_omega_ddd_d+d_omega_ddd_d.swap_adjacent_indices(2,3,4)+contr1-contr2+contr3-contr4

Check that $R_{(a)(b)(c)(d)}$ calculated above is equal to $R(e_{(a)},e_{(b)},e_{(c)},e_{(d)})$ :

In [19]:
Riem_tetrad_dddd[:] == Riem_dddd.comp(e)[:]

True

Definition of a tensor field of type (0,4) from the components $R_{(a)(b)(c)(d)}$ with respect to the base induced by the tetrad base, then I show its components with respect to the tetrad base: 

In [20]:
Riem_dddd_ = M.tensor_field(0,4, 'R', antisym = [(0,1),(2,3)])
Riem_dddd_.add_comp(e)[:] = Riem_tetrad_dddd[:]
Riem_dddd_.display_comp(e, only_nonredundant=True)

R_0101 = -2*m/r^3 
R_0202 = m/r^3 
R_0303 = m/r^3 
R_1212 = -m/r^3 
R_1313 = -m/r^3 
R_2323 = 2*m/r^3

I calculate the components of this tensor defined above with respect to the coordinate base:

In [21]:
Riem_dddd_.display_comp(SD.frame(),only_nonredundant=True)

R_t,r,t,r = -2*m/r^3 
R_t,th,t,th = -(2*m^2 - m*r)/r^2 
R_t,ph,t,ph = -(2*m^2 - m*r)*sin(th)^2/r^2 
R_r,th,r,th = m/(2*m - r) 
R_r,ph,r,ph = m*sin(th)^2/(2*m - r) 
R_th,ph,th,ph = 2*m*r*sin(th)^2

Check that this tensor is the same tensor as the one calculated at the beginning lowering an index from the riemann tensor provided by Sage:

In [22]:
Riem_dddd_ == Riem_dddd #correct

True

Same check on the components collection with respect to the coordinate base:

In [23]:
Riem_dddd_.comp(SD.frame())[:] == Riem_dddd.comp(SD.frame())[:] #correct

True

## Frame bundle and Ricci rotation coefficents 1-forms as its connection

Initialization of the frame bundle as an abstract vector bundle, and definition of a local frame $\{e_{[fb](a)}\}_{(a)=0,1,2,3}$:

In [24]:
fb = M.vector_bundle(4, 'fb', field='complex') #definition of the astract frame bundle
e_fb = fb.local_frame('e')

The connection of an arbitrary vector bundle $V$ is a map of the form $D_{V}: TM \times V \to V$ that takes a section of the tangent bundle and a section of the vecor bundle $V$ and gives a section of the vector bundle $V$.
The map $D_{V}(-,e_{[V]i})=\Gamma^{j}_{\ \ \ \ i}(-)e_{[V]j}$ ($\{e_{[V]i}\}_{i=0,1,2,3}$ local frame on $V$) defines naturally a collection of 1-forms $\Gamma^{j}_{\ \ \ \ i}$ named "connection 1-forms". In our case $V$ is the frame bundle defined before.

In this case we are going to replace the indices $i$ and $j$ with indices in tetrad notation $(a)$ and $(b)$ to label the collection of 1-forms and we are going to use the symbol $\omega$ insted of $\Gamma$ (later it will be clear why).

Definition of the frame bundle connection 1-forms $\omega^{(a)}_{ \ \ \ \ (b)}$ from the Ricci rotation coefficents $\omega^{(a)}_{ \ \ \ \ (b)}= \omega^{(a)}_{ \ \ \ \ (b)\mu}dx^{\mu}$ :

In [25]:
nabla_fb = fb.bundle_connection('\\nabla') #definition of the connection on the frame bundle
for i in range(0,4):
    for j in range(0,4):
        nabla_fb.set_connection_form(i,j)[:] = nabla.connection_form(i,j,e) #setting the connection 1-forms as the connection evaluated on the tetrad

Display of the compoents of the connection:

In [26]:
nabla_fb.display()

connection (0,1) of bundle connection \nabla w.r.t. Local frame (fb|_M, (e_0,e_1,e_2,e_3)) = m/r^2 dt 
connection (1,0) of bundle connection \nabla w.r.t. Local frame (fb|_M, (e_0,e_1,e_2,e_3)) = m/r^2 dt 
connection (1,2) of bundle connection \nabla w.r.t. Local frame (fb|_M, (e_0,e_1,e_2,e_3)) = -sqrt(-2*m + r)/sqrt(r) dth 
connection (1,3) of bundle connection \nabla w.r.t. Local frame (fb|_M, (e_0,e_1,e_2,e_3)) = -sqrt(-2*m + r)*sin(th)/sqrt(r) dph 
connection (2,1) of bundle connection \nabla w.r.t. Local frame (fb|_M, (e_0,e_1,e_2,e_3)) = sqrt(-2*m + r)/sqrt(r) dth 
connection (2,3) of bundle connection \nabla w.r.t. Local frame (fb|_M, (e_0,e_1,e_2,e_3)) = -cos(th) dph 
connection (3,1) of bundle connection \nabla w.r.t. Local frame (fb|_M, (e_0,e_1,e_2,e_3)) = sqrt(-2*m + r)*sin(th)/sqrt(r) dph 
connection (3,2) of bundle connection \nabla w.r.t. Local frame (fb|_M, (e_0,e_1,e_2,e_3)) = cos(th) dph

Check of the frame bundle covariant derivative $D_{fb} (e_{(0)},A) = e_{(0)}^{\ \ \ \ \mu}A^{(a)}_{\ \ \ \ |\mu}e_{[fb](a)}$ of a frame bundle section $A = A^{(a)}e_{[fb](a)}$, where $A^{(a)}_{\ \ \ \ |\mu} = A^{(a)}_{\ \ \ \ ,\mu} + \omega^{(a)}_{\ \ \ \ (b) \mu} A^{(b)}$ denotes the "intrinsic derivative" of Chandrasekar:

In [55]:
A0 = function("A0", nargs = 4)
A1 = function("A1", nargs = 4)
A2 = function("A2", nargs = 4)
A3 = function("A3", nargs = 4)


A0_ = M.scalar_field({SD: A0(r,t,th,ph)}, name = '\\A^{(1)}')
A1_ = M.scalar_field({SD: A1(r,t,th,ph)}, name = '\\A^{(2)}')
A2_ = M.scalar_field({SD: A2(r,t,th,ph)}, name = '\\A^{(3)}')
A3_ = M.scalar_field({SD: A3(r,t,th,ph)}, name = '\\A^{(4)}')

A_prova = fb.section({e_fb: [A0_,A1_,A2_,A3_]}, "A")
nabla_fb(e[0],A_prova).display()

-(r^2*d(A0)/dt + m*A1(r, t, th, ph))*sqrt(-2*m + r)/((2*m*r - r^2)*sqrt(r)) e_0 - (r^2*d(A1)/dt + m*A0(r, t, th, ph))*sqrt(-2*m + r)/((2*m*r - r^2)*sqrt(r)) e_1 + sqrt(r)*d(A2)/dt/sqrt(-2*m + r) e_2 + sqrt(r)*d(A3)/dt/sqrt(-2*m + r) e_3

So as we can see, if we interpret $A^{(a)}$ as the components of a section of the frame bundle instead of the components of a vector field of the tangent bundle with respect to the tetrad frame we can see Chandrasekar's intrinsic derivative as the frame bundle covariant derivative of that section of the frame bundle with respect to the tetrad vector fields.